#### Here we show how to search and index (with also respect to access levels). We make use of the index created in lesson2

In [2]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import QueryAnswerType, QueryCaptionType, QueryType, VectorizedQuery
import numpy as np
from openai import AzureOpenAI
from azure.search.documents import SearchClient
import os

In [4]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [5]:
def generate_embeddings(text, client):
    return client.embeddings.create(input=[text], model=os.getenv("AZURE_ADA3_DEPLOYMENT_NAME")).data[0].embedding


In [7]:
query = "which is the first document"
query_embeddings = np.array(generate_embeddings(query, client)).tolist()
azure_vector_query = [
    VectorizedQuery(
        vector=query_embeddings,
        k_nearest_neighbors=3,
        fields='content_embeddings',
    )
]

In [10]:
search_client = SearchClient(endpoint=os.getenv('AI_SEARCH_ENDPOINT'), index_name='dummy-index', credential=AzureKeyCredential(os.getenv('AI_SEARCH_KEY')))
retrieved_results =search_client.search(
    search_text=query,
    vector_queries=azure_vector_query,
    select=[
        "title",
        "content",
        "security_groups"
    ],
    top=5,
    query_language='en-US',
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
)

In [11]:
for result in retrieved_results:
    print(result)

{'title': 'chunk_1.txt', 'security_groups': ['Owners', 'Members'], 'content': 'This is the first dummy chunk of text.', '@search.score': 0.03333333507180214, '@search.reranker_score': 1.728476643562317, '@search.highlights': None, '@search.captions': [<azure.search.documents._generated.models._models_py3.QueryCaptionResult object at 0x00000234E2080FA0>]}
{'title': 'EmperorPenguins.txt', 'security_groups': ['Owners', 'Members'], 'content': 'This is the second dummy chunk of text.', '@search.score': 0.016393441706895828, '@search.reranker_score': 1.5406079292297363, '@search.highlights': None, '@search.captions': [<azure.search.documents._generated.models._models_py3.QueryCaptionResult object at 0x00000234E2082EF0>]}
{'title': 'RedPanda.txt', 'security_groups': ['Owners'], 'content': 'This is the third dummy chunk of text.', '@search.score': 0.016129031777381897, '@search.reranker_score': 1.510123610496521, '@search.highlights': None, '@search.captions': [<azure.search.documents._generat

## Do a search with respect to access levels

In [16]:
search_client = SearchClient(endpoint=os.getenv('AI_SEARCH_ENDPOINT'), index_name='dummy-index', credential=AzureKeyCredential(os.getenv('AI_SEARCH_KEY')))
retrieved_results =search_client.search(
    search_text=query,
    vector_queries=azure_vector_query,
    select=[
        "title",
        "content",
        "security_groups"
    ],
    top=5,
    query_language='en-US',
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="my-semantic-config",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    filter = "security_groups/any(g: g eq 'Members')"
)

In [17]:
for result in retrieved_results:
    print(result)

{'title': 'chunk_1.txt', 'security_groups': ['Owners', 'Members'], 'content': 'This is the first dummy chunk of text.', '@search.score': 0.03333333507180214, '@search.reranker_score': 1.728476643562317, '@search.highlights': None, '@search.captions': [<azure.search.documents._generated.models._models_py3.QueryCaptionResult object at 0x00000234E2254550>]}
{'title': 'EmperorPenguins.txt', 'security_groups': ['Owners', 'Members'], 'content': 'This is the second dummy chunk of text.', '@search.score': 0.016393441706895828, '@search.reranker_score': 1.5406079292297363, '@search.highlights': None, '@search.captions': [<azure.search.documents._generated.models._models_py3.QueryCaptionResult object at 0x00000234E2254100>]}
